In [1]:
# Etapa 1: Query SQL para tab_silver_dim_produto_calibracao

from pyspark.sql.functions import col, when, to_date, lit

# Rebase para datas antigas
spark.conf.set("spark.sql.legacy.parquet.datetimeRebaseModeInRead", "CORRECTED")
spark.conf.set("spark.sql.legacy.parquet.datetimeRebaseModeInWrite", "CORRECTED")

# Caminho físico do lakehouse de destino
path_destino = "abfss://ws_contabilidade@onelake.dfs.fabric.microsoft.com/lk_contabilidade.Lakehouse/Tables/tab_silver_dim_produto_calibracao"

tproduto = spark.read.format("delta").load(
  "abfss://ws_dados_central@onelake.dfs.fabric.microsoft.com/lk_GestaoEstoqueComprasFaturamento.Lakehouse/Tables/tproduto"
)
tproduto.createOrReplaceTempView("tproduto")

tnumseriecompl = spark.read.format("delta").load(
  "abfss://ws_dados_central@onelake.dfs.fabric.microsoft.com/lk_GestaoEstoqueComprasFaturamento.Lakehouse/Tables/tnumseriecompl"
)
tnumseriecompl.createOrReplaceTempView("tnumseriecompl")

tnumserie = spark.read.format("delta").load(
  "abfss://ws_dados_central@onelake.dfs.fabric.microsoft.com/lk_GestaoEstoqueComprasFaturamento.Lakehouse/Tables/tnumserie"
)
tnumserie.createOrReplaceTempView("tnumserie")

tprdcompl = spark.read.format("delta").load(
  "abfss://ws_dados_central@onelake.dfs.fabric.microsoft.com/lk_GestaoEstoqueComprasFaturamento.Lakehouse/Tables/tprdcompl"
)
tprdcompl.createOrReplaceTempView("tprdcompl")


df_resultado = spark.sql("""
SELECT DISTINCT 
	t.IDPRD,
	t.CODIGOPRD,
	t.DESCRICAO,
	t2.CODCOLIGADA,
	t2.NUMSERIE,
	t2.CODFILIAL,
	t2.CODLOC,
	t2.DATAFABRICACAO,
	t3.RN12,
	t3.NRLOTE,
	t4.MANUTPREVENTIVA,
	t4.PRODUTOCALIBRAVEL,
	t3.DTCALIBRACAO AS DTCALIBRACAO_NUMSERIE,
	t3.DTPROXCALIBRACAO AS DTPROXCALIBRACAO_NUMSERIE,
	t3.NRCERTIFICADOCALI AS NRCERTIFICADOCALI_NUMSERIE,
	t3.CONDICAOCALIBRACAO  AS CONDICAOCALIBRACAO_NUMSERIE,
	t3.OBSCALIBRACAO AS OBSCALIBRACAO_NUMSERIE,
	t4.DTCALIBRACAO, 
	t4.DTPROXCALIBRACAO,
	t4.CONDICAOCALIBRACAO,
	t4.OBSCALIBRACAO 
FROM tproduto t
LEFT JOIN tnumserie t2 
	ON t.CODCOLPRD = t2.CODCOLIGADA AND t.IDPRD = t2.IDPRD
LEFT JOIN tnumseriecompl t3 
	ON t2.CODCOLIGADA = t3.CODCOLIGADA AND t2.IDPRD = t3.IDPRD AND t2.NUMSERIE = t3.NUMSERIE 
LEFT JOIN tprdcompl t4
    ON t.CODCOLPRD = t4.CODCOLIGADA AND t.IDPRD = t4.IDPRD 
WHERE t4.PRODUTOCALIBRAVEL = 'SIM' OR t4.MANUTPREVENTIVA = 'SIM'
""")

# ================================
# Etapa 3: Correção de datas inválidas (mesma regra)
# ================================
data_minima = to_date(lit("1900-01-01"))
data_padrao = to_date(lit("2000-01-01"))

df_corrigido = (
    df_resultado
      .withColumn("DATAFABRICACAO", when(col("DATAFABRICACAO") < data_minima, data_padrao).otherwise(col("DATAFABRICACAO")))
      .withColumn("DTCALIBRACAO", when(col("DTCALIBRACAO") < data_minima, data_padrao).otherwise(col("DTCALIBRACAO")))
      .withColumn("DTPROXCALIBRACAO", when(col("DTPROXCALIBRACAO") < data_minima, data_padrao).otherwise(col("DTPROXCALIBRACAO")))
)

# Salva no Lakehouse de destino usando caminho físico
df_resultado.write.format("delta").mode("overwrite").save(path_destino)


StatementMeta(, 805ecf9e-6b20-42aa-8455-ca971cb472af, 3, Finished, Available, Finished)